## Predicting Market Outcomes: Utilizes Data Research and DataFrames to Cluster Data

Utilized Traits:
* Betting Data (Investment Amounts, Frequency, Wallets)
* Outcome Data (Distribution, Winner)
* Market Details (Category, Date, Time Length)

Open DataFrames

In [1]:
import pandas as pd

marketOutcomes = pd.read_csv('data/silver/marketOutcomes.csv')
markets = pd.read_csv('data/silver/markets_with_ai_categories.csv')

Obtain Addresses

In [2]:
addresses = markets['marketMakerAddress']

In [ ]:
import os
from dotenv import load_dotenv
import requests
import pprint
load_dotenv()
from datetime import datetime, timedelta
POLYGON_API_KEY = os.getenv("POLYGONSCAN_API_KEY")
print(POLYGON_API_KEY)
def time_decoder(address: str, iter: int = None, sz: str = None, hundredPlusBuys = None, lessThanHundredBuys = None): 
    dictForAddys = {}
    if sz is not None:
        if sz == 'large':
            dictForAddys = hundredPlusBuys[iter]
        elif sz == 'small':
            dictForAddys = lessThanHundredBuys[iter]
    
    else:
        dictForAddys = pd.read_csv(f'data/bronze/contract_buy_{address}.csv')
        
    people = [address for address in dictForAddys['buyer']]
    people = [addy.lower() for addy in people]
    url = (
      f'https://api.polygonscan.com/api'
      f'?module=account'
      f'&action=tokentx'
      f'&address={address}'
      f'&startblock=0'
      f'&endblock=99999999'
      f'&apikey={POLYGON_API_KEY}'
    )
    response = requests.get(url)

    pprint.pprint(response.json())

    transactionDict = response.json()

    timeDict = {}
    for addy in transactionDict.get('result', []):
      if addy.get('from', '').lower() in people:
            try:
               timeDict[addy.get('from','').lower()].append(addy.get('timeStamp'))
            except KeyError:
               timeDict[addy.get('from', '').lower()] = [addy.get('timeStamp')]

    from datetime import datetime

    for addy in timeDict:
        timeDict[addy] = list(map(lambda x: datetime.fromtimestamp(int(x)), timeDict[addy]))
    
    
    allTimes = [ts for ts_list in timeDict.values() for ts in ts_list]

    min_timestamp = min(allTimes)

    for addy in timeDict:
        timeDict[addy] = [
            timedelta(seconds=(ts - min_timestamp))
            for ts in timeDict[addy]
        ]
    return timeDict

JCBJI6AJS9E71ZVH58E8IT9E8JZBCCRQEE


In [17]:
listOfTimeDicts = [time_decoder(addy) for addy in addresses]

{'message': 'OK',
 'result': [{'blockHash': '0x68efaf30b371c490a08d0062d41e64f59fa5d93dc2d126f8d218b3eadf085da1',
             'blockNumber': '10035280',
             'confirmations': '60181854',
             'contractAddress': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174',
             'cumulativeGasUsed': '478069',
             'from': '0x404472145fef687e106982497d7f9b6a4634f94c',
             'gas': '880619',
             'gasPrice': '2500000000',
             'gasUsed': '478069',
             'hash': '0x0a9be77c42e13ad0ec5a3445c5905fa8df271cda47c7ff766860623210f6c9a4',
             'input': 'deprecated',
             'nonce': '119562',
             'timeStamp': '1611601449',
             'to': '0x3c872a803eff969139f8a7f2f14121feb6ade5f6',
             'tokenDecimal': '6',
             'tokenName': 'USD Coin (PoS)',
             'tokenSymbol': 'USDC.e',
             'transactionIndex': '0',
             'value': '11000000'},
            {'blockHash': '0x68efaf30b371c490a08d0062d41e64

TypeError: 'datetime.datetime' object cannot be interpreted as an integer